In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import imodelsx.process_results
from collections import defaultdict
import numpy as np
import viz
import transformers
import sys
import tprompt.utils
from os.path import join
import datasets
from typing import Dict, List
from sklearn.tree import plot_tree

from abc import ABC, abstractmethod
import logging
import math
import random
import imodels
import imodelsx.util
import imodelsx.metrics
import numpy as np
import tprompt.utils
from scipy.special import softmax
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import torch.cuda
import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.base import BaseEstimator, ClassifierMixin
sys.path.append('../experiments/')

from tprompt.api import TreePromptClassifier

In [ ]:
dset_train = datasets.load_dataset('rotten_tomatoes')['train']
dset_train = dset_train.select(np.random.choice(
    len(dset_train), size=100, replace=False))
dset_val = datasets.load_dataset('rotten_tomatoes')['validation']
dset_val = dset_val.select(np.random.choice(
    len(dset_val), size=100, replace=False))

In [ ]:
prompts = [
    "This movie is",
    " Positive or Negative? The movie was",
    " The sentiment of the movie was",
    " The plot of the movie was really",
    " The acting in the movie was",
]
verbalizer = {0: " Negative.", 1: " Positive."}
# m = TreePromptClassifier(
#     checkpoint="gpt2",
#     prompts=prompts,
#     verbalizer=verbalizer,
#     cache_prompt_features_dir=None, #'cache_prompt_features_dir/gp2',
# )
# m.fit(dset_train["text"], dset_train["label"])

In [90]:
print(model.transformer)

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)


In [106]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
import torch

# Initialize the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2').eval()
inputs = tokenizer("Hi, my name is", return_tensors="pt")

In [107]:
logits_orig = model(**inputs).logits


def modify_activations(module, inputs, outputs):
    # Define the hook function
    print(module)
    return outputs * 3  # * 0.01


# Add the hook to the first transformer layer
hook = model.transformer.drop.register_forward_hook(modify_activations)
# hook = model.transformer.h[3].register_forward_hook(change_activations)
# Use the model

logits_modified = model(**inputs).logits
most_probable_tokens = torch.topk(logits_modified, k=10, dim=-1)
print('\n'.join([tokenizer.decode(x)
      for x in most_probable_tokens.indices[0, -1]]))


# Don't forget to remove the hook when you're done
hook.remove()

Dropout(p=0.1, inplace=False)
 not
 John
 a
 James
 "
 the
 J
 David
 Chris
,


In [108]:
logits_orig - logits_modified

tensor([[[ -3.1805,  -3.0734,  -3.0371,  ...,  -1.8651,  -2.2625,  -2.4610],
         [-23.9639, -25.1146, -25.5593,  ..., -26.0768, -26.4288, -23.8662],
         [-16.8143, -17.9152, -18.2056,  ..., -19.3295, -20.3710, -18.1819],
         [ 11.9138,  12.9924,  12.5728,  ...,  12.7233,  10.6170,  12.0346],
         [  4.2367,   4.4423,   3.6835,  ...,   4.4055,   4.1055,   5.9551]]],
       grad_fn=<SubBackward0>)